In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()
import pyLDAvis
import pyLDAvis.lda_model
     
client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_attraction = db['attraction_place_2023']
collection_tour = db['tour_contents']
data_tour = collection_tour.find()
df_tour = pd.DataFrame(data_tour)
df_tour

,_id,review_text,지역
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원
...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주


In [27]:
def split(text):
    clean_text = []
    for word in text.split():
        if "기자" not in word:
            clean_text.append(word)
    return " ".join(clean_text)

In [28]:
df_tour.loc[:,'review_text'] = df_tour['review_text'].apply(split)
df_tour.query("review_text.str.contains('자단')")

,_id,review_text,지역


In [ ]:
stopwords = ['남해','대전','대전','공주','안동','나주','바퀴','보령','고령','신안','특산','공항','곡성','신상','사람','위치','가지','진도','장동','정리','도미','고창','모음','장성','시작','남원','가격','화순','국립','베스트','오토','레이스','후기','피아','중앙','공모전','태안','성주','봉화','남도','목포','군산','구례','순천','문경','완주','청주','청도','여수','울주군','포구','두기','거리','강진','뉴스','방법','통영','부여','한밭수목원','거제','서쪽','동쪽','양산','하동','포항','진주','우도','서귀포','당신', '대관령','속초','특별자치도', '삼척','영월','원주','화천','고성','양양','춘천','양구','철원','홍천','태백','달성군','구석구석','블로그','여행','여행지','경기','여주','서구','현황', '장생포','양평','가창','남평문씨','파주','영상','소개','지역','명소','요즘','마을','달성','한눈','카드','추천','수성','안산','기회','횡성','문진','발생','코스','현풍','문진','만끽']

In [29]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt = Okt()
def Tokenizer(raw, pos=["Noun"], stopword=stopwords):
    word_list = []
    for word, tag in okt.pos(raw,norm=True,stem=True):
        if len(word) > 1 and tag in pos and word not in stopword:
            if mecab.pos(word)[0][1] in ["NNG"]:
                if word == '화관':
                    word = '문화관'
                word_list.append(word)
    return " ".join(word_list)

In [30]:
df_tour.loc[:,'review_text_clean']=df_tour['review_text'].apply(Tokenizer)

In [31]:
df_tour

,_id,review_text,지역,review_text_clean
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원,이벤트 취향 저격 이벤트 로드
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원,소리 겨울 아쉬움
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원,이불 태평성대 봉황 눈앞
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원,열흘 통리
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원,걸음 걸음 바다로
...,...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주,전국 맛집
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주,바캉스 당일치기
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주,패류 화석 산지
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주,한림 용암동굴 지대


In [32]:
from sklearn.model_selection import train_test_split
df_tour_train,df_tour_test = train_test_split(df_tour,test_size=0.4,random_state=42)

In [33]:
df_tour_train

,_id,review_text,지역,review_text_clean
14723,65ee6316a64dee66ed301e2a,[블로그 울산 남구 아이스팩 전용 수거함 어디에 있을까?,울산,아이스팩 전용 수거
27198,65ee670ba64dee66ed303175,[가을여행] 전주 당일치기 여행 BEST10 (feat. 덕진공원팸투어) /가을 단풍 여행지추천,전북,가을 전주 당일치기 진공 투어 가을 단풍
35006,65ee6a64c528b55b0ae79e58,"대전 유성구 관평동 맛집, 감동적인 맛과 가격 '매봉 쌀국수와 소바'",대전,맛집 감동 쌀국수
8213,65ee60fc901db79521bc991b,역사를 따라가는 DMZ 로드,경기,역사 로드
24324,65ee664fc528b55b0ae78d7a,"아브라다카브라라는 주문을 외워보고 싶은, '대전 보라매공원크리스마스 풍경'",대전,주문 보라매 공원 풍경
...,...,...,...,...
11284,65ee61dfc528b55b0ae78714,"현풍 가볼만한 곳 :: 충, 효, 열, 삼강의 고장, 현풍 솔례 마을을 찾아서 - 달성군 블로그 김대식",대구,삼강 고장 솔례
44732,65ee9860c27eda916fc74677,가볼만한 제주 오름 백약이오름!,제주,백약
38158,65ee6d15c528b55b0ae7aaa8,[2021년 9월 핫이슈] 서울시 월별 인기 콘텐츠 리뷰 TOP5,서울,핫이슈 월별 인기 콘텐츠
860,65ee5e90901db79521bc8ad5,정선 만항재의 가을 풍경! 맑고 상쾌한 가을 숲의 기운을 느껴봐요~,강원,가을 풍경 가을 기운


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(use_idf=True,max_df=0.95,min_df=2,ngram_range=(1,2))
features = tfidfVectorizer.fit_transform(df_tour_train["review_text_clean"])
features.toarray()[:2]
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['가객', '가게', '가게 선정', ..., '힐링캠프', '힘겨루기', '힘겨루기 대회'], dtype=object)

### component 10

In [37]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model_10 = LatentDirichletAllocation(n_components=10, random_state=111) #인스턴스화 #n_components 토픽의 갯수
lda_model_10.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model_10.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:20])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_10 = [['Topic1', 'Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8','Topic9','Topic10',],topics_list]
df_topics_keywords_10 = pd.DataFrame(topics_list_add_10)
df_topics_keywords_10=df_topics_keywords_10.T
df_topics_keywords_10

,0,1
0,Topic1,공원 벚꽃 겨울 체험 산책 하늘 랜선 전망대 봄꽃 힐링 유적지 아이 커피 기념 방문 순례 자연휴양림 새해 돼지 광장
1,Topic2,어린이 도서관 공원 아이 어린이 공원 가을 수국 전망 계절 억새 랜선 맛집 숲길 더위 물놀이 독서 겨울 축제 일출 산책
2,Topic3,가을 공원 생태 여름 풍경 힐링 단풍 생태 공원 산책로 호텔 해변 가족 겨울 휴가 가을 풍경 자연 탐방 여름 휴가 산책 습지
3,Topic4,가족 사진 해수욕장 드라이브 공원 사찰 맛집 등대 정원 도서관 철새 아이 디저트 축제 동부 놀이터 사랑 랜선 생활 대학교
4,Topic5,박물관 축제 등산 유채꽃 청년 이야기 고래 볼거리 시간 고택 벽화 전시 문화 벚꽃 산책 갤러리 세계 시립 설경 현장
5,Topic6,관광지 실내 수목원 박물관 분위기 아이 산행 안심 센터 근처 맛집 로드 동백꽃 서포터즈 전시회 지원 공원 기장 실내 관광지 여름
6,Topic7,산책 공원 호수 호수 공원 문화 공간 가을 여유 당일치기 복합 도시 투어 문화 공간 힐링 미식 풍경 근교 가을 산책 인생 문화재
7,Topic8,시장 전통 자연 자전거 전통 시장 매력 맛집 국내 동굴 폭포 예술 구경 힐링 먹거리 로컬 지금 하루 향교 라이딩 숙소
8,Topic9,데이트 맛집 이색 장미 도로 감성 해안 미술관 일몰 아트 노을 해안 도로 여름 공원 체험 청소년 마음 포토 센터 벚꽃
9,Topic10,맛집 역사 문화 공원 파크 체육 테마 투어 골목 관광 평화 랜선 야경 체험 테마 파크 체육 공원 역사 문화 아이 도심 안전


In [39]:
import pyLDAvis
import pyLDAvis.lda_model
vis_10 = pyLDAvis.lda_model.prepare(lda_model_10,features,tfidfVectorizer)
vis_10
pyLDAvis.enable_notebook()
components_10_display = pyLDAvis.display(vis_10)
components_10_display

### component 3

In [40]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model_3 = LatentDirichletAllocation(n_components=3, random_state=111) #인스턴스화 #n_components 토픽의 갯수
lda_model_3.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model_3.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:20])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_3 = [['Topic1', 'Topic2','Topic3'],topics_list]
df_topics_keywords_3 = pd.DataFrame(topics_list_add_3)
df_topics_keywords_3=df_topics_keywords_3.T
df_topics_keywords_3

,0,1
0,Topic1,공원 산책 아이 체험 벚꽃 역사 겨울 박물관 도서관 호수 실내 공간 랜선 전망대 힐링 매력 청년 호수 공원 자연 평화
1,Topic2,맛집 어린이 공원 랜선 축제 야경 숲길 사진 벽화 자전거 아이 감성 도서관 어린이 공원 도심 골목 노을 등산 문화 수국
2,Topic3,가을 여름 풍경 공원 생태 힐링 가족 문화 자연 시장 단풍 산책 맛집 산책로 해수욕장 해변 드라이브 전통 탐방 겨울


In [41]:
import pyLDAvis
import pyLDAvis.lda_model
vis_3 = pyLDAvis.lda_model.prepare(lda_model_3,features,tfidfVectorizer)
vis_3
pyLDAvis.enable_notebook()
components_3_display = pyLDAvis.display(vis_3)
components_3_display

### component 5

In [42]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model_5 = LatentDirichletAllocation(n_components=5, random_state=111) #인스턴스화 #n_components 토픽의 갯수
lda_model_5.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model_5.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:20])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_5 = [['Topic1', 'Topic2','Topic3','Topic4','Topic5'],topics_list]
df_topics_keywords_5 = pd.DataFrame(topics_list_add_5)
df_topics_keywords_5=df_topics_keywords_5.T
df_topics_keywords_5

,0,1
0,Topic1,공원 산책 아이 벚꽃 겨울 도서관 실내 체험 평화 매력 분위기 하늘 전망대 호수 랜선 커피 체육 힐링 박물관 호수 공원
1,Topic2,공원 단풍 랜선 맛집 어린이 수국 가을 아이 숲길 감성 체험 도심 어린이 공원 이벤트 지금 근처 억새 전망 산책 노을
2,Topic3,가을 공원 여름 풍경 생태 힐링 산책 가족 산책로 자연 생태 공원 겨울 맛집 해수욕장 관광지 탐방 휴가 산책길 물놀이 숙소
3,Topic4,맛집 역사 드라이브 공원 투어 해변 사진 놀이터 일몰 골목 인생 데이트 문화 체험 아이 촬영 겨울 랜선 벽화 여름
4,Topic5,문화 박물관 축제 시장 전통 전시 예술 유채꽃 이야기 정원 공원 야경 미술관 등산 전통 시장 청년 아트 역사 자전거 공간


In [43]:
import pyLDAvis
import pyLDAvis.lda_model
vis_5 = pyLDAvis.lda_model.prepare(lda_model_5,features,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_5_display = pyLDAvis.display(vis_5)
components_5_display

### component 7

In [44]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model_7 = LatentDirichletAllocation(n_components=7, random_state=111) #인스턴스화 #n_components 토픽의 갯수
lda_model_7.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model_7.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:20])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_7 = [['Topic1', 'Topic2','Topic3','Topic4','Topic5','Topic6','Topic7'],topics_list]
df_topics_keywords_7 = pd.DataFrame(topics_list_add_7)
df_topics_keywords_7=df_topics_keywords_7.T
df_topics_keywords_7

,0,1
0,Topic1,벚꽃 공원 아이 도서관 겨울 랜선 자연 체험 매력 박물관 하늘 전망대 산책 휴식 힐링 봄꽃 디저트 서원 어린이 세계
1,Topic2,공원 어린이 수국 물놀이 아이 어린이 공원 체험 감성 맛집 도심 숲길 여름 랜선 축제 억새 청소년 전망 도서관 계절 가을
2,Topic3,가을 공원 생태 풍경 시장 단풍 힐링 생태 공원 산책로 여름 가족 겨울 전통 전통 시장 자연 매화 해수욕장 산책 마음 가을 풍경
3,Topic4,맛집 해변 드라이브 공원 사진 노을 골목 놀이터 일몰 겨울 칼국수 체험 사랑 음식 일출 해수욕장 역사 버스 인생 간직
4,Topic5,박물관 축제 정원 파크 야경 등산 테마 유채꽃 문화 장미 폭포 자전거 공원 고래 전시 테마 파크 볼거리 동굴 산책 시간
5,Topic6,여름 관광지 데이트 실내 수목원 관광 평화 청년 이야기 맛집 박물관 아이 예술 미술관 분위기 근교 근처 로드 도로 안심
6,Topic7,공원 산책 문화 역사 호수 호수 공원 투어 힐링 공간 벽화 가을 체육 커피 여유 센터 체육 공원 도시 역사 문화 미식 도심


In [45]:
import pyLDAvis
import pyLDAvis.lda_model
vis_7 = pyLDAvis.lda_model.prepare(lda_model_7,features,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_7_display = pyLDAvis.display(vis_7)
components_7_display

## 결론
- 다양한 토픽으로 분류하고 싶었지만, 여러개로 나눌려고 하면 토픽별로 겹쳐서 유의미하지 않다고 판단.
- 학습 데이터에 토픽을 라벨링하여 준지도학습 진행 시도